In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from wall_analysis import parse_experiments
import seaborn as sns
from plotting_tools import save_fig

def plot_plositions(row, min_time=None, max_time=None, max_dist=None):
    positions_cm = row.positions[:, :3] * 100
    fig, axs = plt.subplots(1, 3) 
    fig.set_size_inches(10, 3.3)
    fig.suptitle(row.appendix, y=1.0)
    
    mask_time = np.ones_like(row.seconds, dtype=np.bool)
    if min_time is not None:
        mask_time = (row.seconds > min_time) 
    if max_time is not None:
        mask_time = mask_time & (row.seconds < max_time)
        
    time = row.seconds[mask_time]
    
    #axs[0].plot(x=positions_cm[:, 0], y=positions_cm[:, 1], color=colors())
    sns.scatterplot(x=positions_cm[mask_time, 0], y=positions_cm[mask_time, 1], 
                    hue=time, ax=axs[0], linewidth=0, 
                    #size=positions_cm[:, 2],
                    palette='inferno')
    axs[0].set_xlabel('x [cm]')
    axs[0].set_ylabel('y [cm]')
    axs[0].axis('equal')
    axs[0].legend(loc='lower right', title='time [s]')

    axs[1].plot(time, positions_cm[mask_time, 0], label='x')
    axs[1].plot(time, positions_cm[mask_time, 1], label='y')
    axs[1].plot(time, positions_cm[mask_time, 2], label='z')
    axs[1].set_xlabel('time [s]')
    axs[1].set_ylabel('movement [cm]')
    if max_dist is not None:
        axs[1].set_ylim(-max_dist, max_dist)
    axs[1].legend(loc='lower right')

    axs[2].plot(time, row.positions[mask_time, 3], label='yaw')
    axs[2].set_ylabel('yaw [deg]')
    axs[2].set_xlabel('time [s]')
    axs[2].set_ylim(-20, 20)
    axs[2].legend(loc='lower right')
    plt.tight_layout()
    return fig, axs

def plot_audio(row, mic_idx=0):
    all_frequencies = np.unique(row.frequencies_matrix)
    spec = row.spectrogram[:, mic_idx, :]
    spec[spec == 0] = np.nan
    total = np.nanmean(np.abs(spec), axis=1)
    
    label = str(f"{row.appendix}").replace('_', '')
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    ax.set_title(f'spectrogram of mic{mic_idx}, appendix {row.appendix}')
    
    # mark too long measurements gray
    max_diff = 1
    diff = row.seconds[1:] - row.seconds[:-1]
    indices = np.where(diff>max_diff)[0]
    endings = row.seconds[:-1][indices]
    diff_average = np.mean(diff[diff<max_diff])
    seconds = row.seconds
    for counter, i in enumerate(indices):
        new_time = seconds[i+counter]+diff_average
        seconds = np.insert(seconds, i+counter+1, new_time)
        spec = np.insert(spec, i+counter+1, np.nan, axis=1)
    
    pcolorfast_custom(ax, seconds, all_frequencies, np.abs(spec))
    
    xticks = np.arange(0, row.seconds.max(), step=5)
    ax.set_xticks(xticks); ax.set_xticklabels(xticks)
    yticks = np.arange((np.round(row.frequencies_matrix.min()//1000)+1)*1000, 
                       row.frequencies_matrix.max(), step=1000)
    ax.set_yticks(yticks); ax.set_yticklabels(yticks)
    ax.set_ylabel('frequency [Hz]')
    ax.set_xlabel('seconds [s]')
    return fig, ax

# 1. Frequency slice

In [ ]:
#exp_name = '2020_12_18_stepper'; appendix = ""; distance = 51
#exp_name = '2020_12_18_flying'; appendix="_new"; distance = 0
#exp_name = '2021_03_01_flying';
#exp_name = '2021_04_30_hover';
exp_name = '2021_05_04_flying';
fname = f'../experiments/{exp_name}/all_data.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    answer = input('Run wall_analysis.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

## 1.1 positions analysis

In [ ]:
from geometry import Context
context = Context.get_crazyflie_setup(dim=2)
context.plot()

In [ ]:
min_time = None
max_time = None
max_dist = None
for i, row in df_total.iterrows():
    fig, axs = plot_plositions(row, min_time, max_time, max_dist)
    save_fig(fig, f'plots/{exp_name}{row.appendix}_movement.pdf')

## 1.2 audio analysis

In [ ]:
from frequency_analysis import add_spectrogram
from plotting_tools import pcolorfast_custom

df_total = df_total.assign(spectrogram=None)
df_total = df_total.apply(add_spectrogram, axis=1)

mic_idx = 0

#maxi = np.nanmax(np.concatenate([*dfs.spectrogram], axis=1))
for i_col, row in df_total.iterrows():
    fig, ax = plot_audio(row, mic_idx=mic_idx)


## 1.3 algorithm performance

In [ ]:
# choose the experiment to analyze
row = df_total.iloc[0] # works well
#row = df_total.iloc[1] # works ok
#row = df_total.iloc[2] # doesn't work

In [ ]:
from calibration import get_calibration_function_median, get_calibration_function_dict

fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
if "newbuzzer" in row.appendix:
    calib_function, calib_freq = get_calibration_function_median(
        exp_name="2021_02_25_wall", mic_type="audio_deck", ax=ax
    )
else:
    calib_function, calib_freq = get_calibration_function_median(
        "2021_04_30_stepper", "audio_deck", ax=ax, #fit_one_gain=True 
    )
#else:
#    calib_function, calib_freq = get_calibration_function_median(
#        exp_name="2021_02_23_wall", mic_type="audio_deck", ax=ax
#    )


In [ ]:
from dataset_parameters import kwargs_datasets
from simulation import get_df_theory
from crazyflie_description_py.experiments import WALL_ANGLE_DEG

azimuth_deg = WALL_ANGLE_DEG

distance_range = [0, 70]
kwargs = kwargs_datasets[exp_name]["audio_deck"]

distances_grid = np.linspace(distance_range[0]+7, distance_range[1],100)
freqs_theo = np.linspace(kwargs["min_freq"], kwargs["max_freq"], 200)

df_matrix_theo = get_df_theory(freqs_theo, distances_grid, azimuth_deg=azimuth_deg, 
                               chosen_mics=[mic_idx])

fig, ax_df = plt.subplots()
xticks = [7, 10, 20, 30, 40, 50, 60, 70]
yticks = np.arange(kwargs["min_freq"], kwargs["max_freq"]+1, step=1000)
pcolorfast_custom(ax_df, distances_grid, freqs_theo, np.log10(df_matrix_theo[0]), cmap='Greys')
ax_df.set_xticks(xticks); ax_df.set_xticklabels(xticks)
ax_df.set_yticks(yticks); ax_df.set_yticklabels(yticks)
ax_df.set_xlabel('distance [cm]')
ax_df.set_ylabel('frequency [Hz]')

In [ ]:
# spec_masked, freqs_masked = data_collector.fill_from_row()
from data_collector import DataCollector
from estimators import DistanceEstimator
from inference import get_probability_bayes
from simulation import get_freq_slice_theory

linestyles = {"measured": "-", "theo": ":", "theo_corr": "-."}
EPS = 1e-10

starting_distance = 60  # [50, 30, 10]
nominal_distances = [60 - i * 20 for i in range(3)]
slice_idx = 0

data_collector = DataCollector(exp_name=exp_name)

fig, ax_distance = plt.subplots()
fig.set_size_inches(10, 5)
ax_distance.set_xlabel("time idx")
ax_distance.set_ylabel("distance [cm]")
ax_distance.axhline(starting_distance, label="starting distance", ls=":")
ax_distance.set_title(f"experiment {row.appendix}")

freqs_calib = np.linspace(
    np.min(row.frequencies_matrix), np.max(row.frequencies_matrix), 50
)
f_calib_all = calib_function(freqs_calib)

fig, ax_df = plt.subplots()
pcolorfast_custom(ax_df, distances_grid, freqs_theo, np.log10(df_matrix_theo[0]), cmap='Greys', n_xticks=10)
ax_df.set_xticks(xticks); ax_df.set_xticklabels(xticks)
ax_df.set_yticks(yticks); ax_df.set_yticklabels(yticks)
ax_df.set_xlabel('distance [cm]')
ax_df.set_ylabel('frequency [Hz]')

for i in range(row.stft.shape[0]):
    signals_f = row.stft[i]
    frequencies = row.frequencies_matrix[i]

    if i == row.stft.shape[0] - 1:
        sweep_complete = True
    else:
        sweep_complete = data_collector.next_fslice_ready(signals_f, frequencies)

    if sweep_complete:
        f_slice, freqs, stds, distances = data_collector.get_current_frequency_slice(
            verbose=False
        )

        fig, ax_total = plt.subplots()
        fig.set_size_inches(10, 5)
        ax_total.set_xlabel("distance [cm]")
        ax_total.set_ylabel("probability")
        ax_total.set_title(f"experiment {row.appendix}")

        fig, axs = plt.subplots(3, row.stft.shape[1])  # , sharey='row')
        fig.set_size_inches(15, 10)
        fig.suptitle(f"up to {data_collector.latest_fslice_time:.1f}s", y=0.9)
        # fig.set_suptitle(f"experiment {row.appendix}")

        # extract valid indices
        valid_idx = (
            (freqs >= min(calib_function.x))
            & (freqs <= max(calib_function.x))
            & ((3800 <= freqs) | (freqs <= 3700))
        )
        freqs = freqs[valid_idx]
        f_calib = calib_function(freqs)
        f_slice = f_slice[:, valid_idx]
        distances = distances[valid_idx]

        distance_corr = starting_distance - distances
        ax_df.scatter(distance_corr, freqs, color=f"C{slice_idx}")

        f_theo = get_freq_slice_theory(
            freqs, distance_cm=nominal_distances[slice_idx], azimuth_deg=azimuth_deg
        ).T  # n_mics x n_freqs
        f_theo_corr = get_freq_slice_theory(
            freqs, distance_cm=distance_corr, azimuth_deg=azimuth_deg
        ).T

        print("treating new frequency slice")

        distance_estimators = {
            "measured": DistanceEstimator(),
            "theo": DistanceEstimator(),
            "theo_corr": DistanceEstimator(),
        }

        for i_mic in range(f_slice.shape[0]):
            axs[0, i_mic].plot(freqs, f_slice[i_mic], label="measured")
            axs[0, i_mic].legend(loc="upper right")

            f_calibrated = f_slice[i_mic] / f_calib[i_mic]

            axs[1, i_mic].plot(freqs, f_calibrated, label="measured", ls="-")
            axs[1, i_mic].plot(freqs, f_theo[i_mic], label="theo", color=f"C6", ls=":")
            axs[1, i_mic].plot(
                freqs, f_theo_corr[i_mic], label="theo corr", color="C7", ls="-."
            )
            axs[0, i_mic].set_title(f"mic{i_mic}")

            # get probability distribution
            valid = ~np.isnan(f_calibrated)
            distances_bayes, proba, diff_bayes = get_probability_bayes(
                f_calibrated[valid],
                freqs[valid],
                mic_idx=i_mic,
                distance_range=distance_range,
                sigma=1.0,
                azimuth_deg=azimuth_deg,
            )
            proba = (proba - np.min(proba) + EPS) / (
                np.max(proba) - np.min(proba) + EPS
            )

            # get probability distribution with theoretical
            valid = np.all(~np.isnan(f_theo), axis=0)
            distances_bayes_theo, proba_theo, diff_bayes_theo = get_probability_bayes(
                f_theo[i_mic, valid],
                freqs[valid],
                mic_idx=i_mic,
                distance_range=distance_range,
                sigma=1.0,
                azimuth_deg=azimuth_deg,
            )
            proba_theo = (proba_theo - np.min(proba_theo) + EPS) / (
                np.max(proba_theo) - np.min(proba_theo) + EPS
            )

            valid = np.all(~np.isnan(f_theo_corr), axis=0)
            (
                distances_bayes_theo_corr,
                proba_theo_corr,
                diff_bayes_theo_corr,
            ) = get_probability_bayes(
                f_theo_corr[i_mic, valid],
                freqs[valid],
                mic_idx=i_mic,
                distance_range=distance_range,
                sigma=1.0,
                azimuth_deg=azimuth_deg,
            )
            proba_theo_corr = (proba_theo_corr - np.min(proba_theo_corr) + EPS) / (
                np.max(proba_theo_corr) - np.min(proba_theo_corr) + EPS
            )
            distance_estimators["measured"].add_distribution(
                diff_bayes * 1e-2, proba, i_mic
            )
            distance_estimators["theo"].add_distribution(
                diff_bayes_theo * 1e-2, proba_theo, i_mic
            )
            distance_estimators["theo_corr"].add_distribution(
                diff_bayes_theo_corr * 1e-2, proba_theo_corr, i_mic
            )

            axs[2, i_mic].plot(distances_bayes, proba, label="measured")
            axs[2, i_mic].plot(distances_bayes_theo, proba_theo, label="theo")
            axs[2, i_mic].plot(
                distances_bayes_theo_corr, proba_theo_corr, label="theo_corr"
            )

        for key, distance_estimator in distance_estimators.items():
            distance_total, proba_total = distance_estimator.get_distance_distribution(
                method="sum", chosen_mics=None, azimuth_deg=azimuth_deg
            )
            ax_total.plot(
                distance_total * 1e2,
                proba_total,
                label=f"{key} d={nominal_distances[slice_idx]}cm",
                # color=f"C{slice_idx}",
                ls=linestyles[key],
            )

        axs[1, i_mic].legend()
        axs[2, i_mic].legend()
        ax_total.legend(loc="upper right")
        ax_distance.plot(distance_corr, label=nominal_distances[slice_idx])

        for i_mic in range(f_calib_all.shape[0]):
            axs[0, i_mic].plot(freqs_calib, f_calib_all[i_mic], label=f"calib", ls=":")
        slice_idx += 1

    data_collector.fill_from_signal(
        signals_f,
        frequencies,
        distance_cm=row.positions[i, 1] * 1e2,
        time=row.seconds[i],
    )

ax_distance.legend(loc="upper right")
ax_total.legend(loc="upper right")

# Notes for this experiment

- apply optimization algorithm including relative distance measurements
- do normalization over all mics rater than per-mic 

# 2. Distance slice

In [ ]:
#exp_name = '2020_12_18_stepper'; appendix = ""; distance = 51
#exp_name = '2020_12_18_flying'; appendix="_new"; distance = 0
#exp_name = '2021_03_01_flying';
#exp_name = '2021_04_30_hover';
exp_name = '2021_05_04_linear';
fname = f'../experiments/{exp_name}/all_data.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    answer = input('Run wall_analysis.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

## 1.1 positions analysis

In [ ]:
starting_positions = {
    '_1': [20, -50, 0, 45],
    '_2': [-20, -50, 0, -45],
    '_3': [0, -50, 0, 0],
    '_4': [-10, -50, 0, -30],
    '_5': [10, -50, 0, 30],
    '_fast1': [-20, -50, 0, -45],
    '_fast2': [20, -50, 0, 45],
    '_fast3': [0, -50, 0, 0],
    '_fast4': [20, -50, 0, 30],
    '_fast5': [-20, -50, 0, -30]
}

In [ ]:
def get_average_angle(positions_rot):
    # angles between -180, 180:
    angles = np.arctan2(positions_rot[:30, 1]-positions_rot[0, 1], positions_rot[:30, 0]-positions_rot[0, 0]) * 180 / np.pi
    
    # convert 120 to 60 etc.
    angles[angles > 90] = 180 - angles[angles > 90]
    return np.median(angles)

def get_gt_angle(row):
    starting_yaw = starting_positions[row.appendix][3]
    approach_angle = starting_yaw + 90
    if approach_angle > 90:
        approach_angle = 180 - approach_angle
    return approach_angle
    

def plot_corrected_positions(row, max_idx=30, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    starting_pose = starting_positions[row.appendix]
    positions_rot = np.empty_like(row.positions)
    for j, pos in enumerate(row.positions):
        total_yaw = starting_pose[3] + pos[3]
        rot = R.from_euler('z', total_yaw, degrees=True)
        pos_rot = starting_pose[:3] + rot.apply(pos[:3]) * 1e2
        positions_rot[j, :] = np.r_[pos_rot, total_yaw]
    
    valid = np.all(~np.isnan(positions_rot), axis=1) & (positions_rot[:, 2] > 35)
    positions_rot = positions_rot[valid, :]
    average_angle = get_average_angle(positions_rot)
    gt_angle = get_gt_angle(row)
    ax.plot(positions_rot[:max_idx, 0], positions_rot[:max_idx, 1], color=f'C{i}', label=f'appendix {row.appendix}, {average_angle:.0f} {gt_angle:.0f}')
    ax.axis('equal')
    return positions_rot

In [ ]:
from scipy.spatial.transform import Rotation as R
min_time = 0
max_time = np.inf
max_dist = 100 # cm
max_idx = 30

fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
for i, row in df_total.iterrows():
    fig, axs = plot_plositions(row, min_time=None, max_time=None, max_dist=None)
    plot_corrected_positions(row, ax=ax)
ax.legend(bbox_to_anchor=[1.0, 1.0], loc='upper left')

## 1.2 audio analysis

In [ ]:
from frequency_analysis import add_spectrogram
from plotting_tools import pcolorfast_custom

df_total = df_total.assign(spectrogram=None)
df_total = df_total.apply(add_spectrogram, axis=1)

mic_idx = 0
#maxi = np.nanmax(np.concatenate([*dfs.spectrogram], axis=1))
for i_col, row in df_total.iterrows():
    pass
    #fig, ax = plot_audio(row, mic_idx=mic_idx)
    
    plt.figure()
    for time_idx in range(row.frequencies_matrix.shape[0]):
        mic_idx = 0
        freqs = row.frequencies_matrix[time_idx, :]
        response = np.abs(row.spectrogram[:, mic_idx, time_idx])
        plt.plot(freqs, response)

In [ ]:
# slow
row = df_total.iloc[0] # works well 45
#row = df_total.iloc[1] # works ok 45
#row = df_total.iloc[2] # works well 90
#row = df_total.iloc[3] # doesn't work 60
#row = df_total.iloc[4] # doesn't work 60
# fast
#row = df_total.iloc[5] # 45

positions_corr = plot_corrected_positions(row, max_idx=-1)
plt.legend()
plt.xlabel('x [cm]')
plt.ylabel('y [cm]')

plt.figure()
plt.plot(row.positions[:, 1]*1e2, positions_corr[:, 1])
plt.xlabel('relative distance [cm]')
plt.ylabel('distance from wall [cm]')

## 1.3 algorithm analysis

In [ ]:
from inference import get_approach_angle_fft
from data_collector import DataCollector

n_mics = 4
gt_angle = get_gt_angle(row)
data_collector = DataCollector(exp_name=exp_name)

for i in range(row.stft.shape[0]):
    signals_f = row.stft[i]
    frequencies = row.frequencies_matrix[i]
    position = row.positions[i]

    if i == row.stft.shape[0] - 1:
        d_slice_ready = True
        print('reached end')
    else:
        d_slice_ready = data_collector.next_dslice_ready(signals_f, frequencies, position*1e2, n_max=50)

    if d_slice_ready:
        d_slices, distances, stds, freqs = data_collector.get_current_distance_slice()
        if len(distances) < 10:
            print(f'skipping last measurement, cause only {len(distances)}')
            break

        fig, axs = plt.subplots(2, n_mics, sharey='row')
        fig.set_size_inches(15, 5)
        for i_mic in range(n_mics):
            d_slice = d_slices[i_mic]
            axs[0, i_mic].set_title(f'mic{i_mic}')
            axs[0, i_mic].scatter(distances, d_slice, color='C0')
            axs[0, i_mic].set_xlabel(f"relative distance [cm]")

            #d_slice -= np.mean(d_slice, axis=1)[None, :]
            valid = ~np.isnan(d_slice)
            #print(distances[valid], d_slice[valid])
            angles, proba = get_approach_angle_fft(
                d_slice=d_slice[valid],
                frequency=np.mean(freqs),
                relative_distances_cm=distances[valid],
                reduced=True
            )
            #EPS = 1e-10
            #proba = (proba - np.min(proba) + EPS) / (np.max(proba) - np.min(proba) + EPS)
            axs[1, i_mic].plot(angles, proba, color='C1')
            axs[1, i_mic].axvline(gt_angle)
            axs[1, i_mic].set_xlabel("approach angle [deg]")
        axs[0, 0].set_ylabel(f"magnitude{row.appendix}")
        axs[1, 0].set_ylabel(f"probability{row.appendix}")

    # only add measurements if the drone is really flying.
    if data_collector.valid_dslice_measurement(position*1e2, signals_f, frequencies):
        data_collector.fill_from_signal(
            signals_f, frequencies, distance_cm=position[1]*1e2, time=row.seconds[i]
        )